In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from object_detection.utils import label_map_util
from object_detection.data_decoders import tf_example_decoder

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models


from tensorflow.keras.layers import Conv2D, Reshape, Concatenate
from tensorflow.keras.models import Model


In [41]:
# Set up the data paths
train_tfrecord_dir = 'ThermalCamera_MLX90640.v3i.tfrecord/train/'
valid_tfrecord_dir = 'ThermalCamera_MLX90640.v3i.tfrecord/valid/'
train_label_map_path = os.path.join(train_tfrecord_dir, 'person_label_map.pbtxt')
valid_label_map_path = os.path.join(valid_tfrecord_dir, 'person_label_map.pbtxt')

# Load the label map
label_map = label_map_util.load_labelmap(train_label_map_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)

# Get the list of TFRecord files
train_tfrecord_files = tf.io.gfile.glob(os.path.join(train_tfrecord_dir, '*.tfrecord'))
valid_tfrecord_files = tf.io.gfile.glob(os.path.join(valid_tfrecord_dir, '*.tfrecord'))

def load_dataset(tfrecord_files, batch_size=32, shuffle_buffer_size=1000):
    """Loads a TFRecord dataset."""
    feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'height': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64, default_value=1),
    'bbox_xmin': tf.io.VarLenFeature(tf.float32),
    'bbox_ymin': tf.io.VarLenFeature(tf.float32),
    'bbox_xmax': tf.io.VarLenFeature(tf.float32),
    'bbox_ymax': tf.io.VarLenFeature(tf.float32),
    'class_labels': tf.io.VarLenFeature(tf.int64),
    # Add other features as needed for your dataset
}


    
    def _parse_tfrecord_fn(example_proto):
        # Parse the TFRecord example
        features = tf.io.parse_single_example(example_proto, feature_description)
        image = tf.image.decode_image(features['image'], channels=3)  # Adjust channels if needed
    
        # Process and preprocess your data here...
        bbox_xmin = tf.sparse.to_dense(features['bbox_xmin'])
        bbox_ymin = tf.sparse.to_dense(features['bbox_ymin'])
        bbox_xmax = tf.sparse.to_dense(features['bbox_xmax'])
        bbox_ymax = tf.sparse.to_dense(features['bbox_ymax'])
    
        # Stack bounding box coordinates into one tensor
        bbox = tf.stack([bbox_ymin, bbox_xmin, 
                         bbox_ymax, bbox_xmax], axis=-1)

        # Convert class labels to one-hot encoding
        num_classes = len(label_map_dict) + 1  # Add one for background class
        class_labels_one_hot = tf.one_hot(tf.sparse.to_dense(features['class_labels']), depth=num_classes)
     
        # Reshape as needed...
        bbox = tf.reshape(bbox, (-1, 4))
        class_labels_one_hot = tf.reshape(class_labels_one_hot , (-1,num_classes))

        ground_truth_annotations={
             'predictions':tf.concat([bbox,class_labels_one_hot],axis=-1)
        }

        return image ,ground_truth_annotations
    return tf.data.TFRecordDataset(tfrecord_files).map(_parse_tfrecord_fn).shuffle(shuffle_buffer_size).batch(batch_size)


# Load the datasets
train_dataset = load_dataset(train_tfrecord_files)
valid_dataset = load_dataset(valid_tfrecord_files)


In [9]:
import tensorflow as tf

# Get the first record file.
record_file = train_tfrecord_files[0]

# Create a dataset from the record file.
dataset = tf.data.TFRecordDataset(record_file)

# Decode the first example.
for raw_record in dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    
    # Print the keys in this example.
    print(example.features.feature.keys())


KeysView({'image/encoded': bytes_list {
  value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000C\000\010\006\006\007\006\005\010\007\007\007\t\t\010\n\014\024\r\014\013\013\014\031\022\023\017\024\035\032\037\036\035\032\034\034 $.\' \",#\034\034(7),01444\037\'9=82<.342\377\333\000C\001\t\t\t\014\013\014\030\r\r\0302!\034!22222222222222222222222222222222222222222222222222\377\300\000\021\010\000\010\000\004\003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\203\204\205\206\207\210\211\212\222\223\224\225\226\227\230\231\232\242\243\244\245\246\247\250\251\252\262\263\264\265\266\267\270\2

In [42]:
# Define the SSD MobileNetV2 model
base_model = MobileNetV2(input_shape=(None, None, 3), include_top=False)
# Add additional layers for object detection
x = base_model.output
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='additional_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='additional_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='additional_conv3')(x)

# Define the prediction head
num_boxes = 1
num_classes = len(label_map_dict) + 1  # Add one for the background class
bbox_predictions = Conv2D(4 * num_boxes, (3, 3), padding='same', name='bbox_predictions')(x)
bbox_predictions = Reshape((-1, 4))(bbox_predictions)
class_predictions = Conv2D(num_classes * num_boxes, (3, 3), padding='same', name='class_predictions')(x)
class_predictions = Reshape((-1, num_classes))(class_predictions)
predictions = Concatenate(axis=-1, name='predictions')([bbox_predictions, class_predictions])


# Create the SSD model|
model = Model(inputs=base_model.input, outputs=predictions)

# Define the loss function and compile the model
loss_fn = Huber(delta=1.0)
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=loss_fn)

# Set up callbacks
checkpoint_callback = ModelCheckpoint("ssd_mobilenetv2_checkpoint.h5", save_best_only=True)
early_stopping_callback = EarlyStopping(patience=10, restore_best_weights=True)

# Train the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'], loss_weights={'predictions': 1.0})



epochs = 50  # Adjust as needed
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
    callbacks=[checkpoint_callback, early_stopping_callback]
)


Epoch 1/50


2023-09-14 22:02:07.717604: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-14 22:02:08.186184: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: image (data type: string) is required but could not be found.
2023-09-14 22:02:08.186209: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: image (data type: string) is required but could not be found.
2023-09-14 22:02:08.186222: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: image (data type: string) is required but could not be found.
2023-09-14 22:02:08.186231: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: image (data type: string) is required but could not

InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  Feature: image (data type: string) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
	 [[IteratorGetNext]]
	 [[model_17/Cast/_8]]
  (1) INVALID_ARGUMENT:  Feature: image (data type: string) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_105859]

['ThermalCamera_MLX90640.v3i.tfrecord/train/person.tfrecord']
['ThermalCamera_MLX90640.v3i.tfrecord/valid/person.tfrecord']


AttributeError: 'NoneType' object has no attribute 'take'

In [31]:

epochs = 50  # Adjust as needed
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
    callbacks=[checkpoint_callback, early_stopping_callback]
)


ValueError: Failed to find data adapter that can handle input: <class 'NoneType'>, <class 'NoneType'>